# Location Cleaning Framework

Braden Riggs A15089134 DSC198

Additional geodata provided by Geonames.org

In [1]:
#For Geocoding with google must install
#! pip install geocoder

In [2]:
#Import packages for cleaning
#!pip install pycountry
#!pip install geopy
import pandas as pd
import numpy as np
import pycountry
from geopy.geocoders import Nominatim
from geopy.geocoders import GeoNames
from geopy.geocoders import Photon
pd.options.display.max_columns = None 

Enter file for standardization:

In [3]:
std_file = "CNCB"

Enter location column names(s):

In [4]:
loc1 = "Location"
loc2 = "none"
loc3 = "none"
loc4 = "none"

Trun on and enter the desired matching strength for matching locations on Geonames:

In [5]:
matching_strength_on = False
matching_strength = 0.8

Run Below Code:

In [6]:
metadata_url = "https://bigd.big.ac.cn/ncov/genome/export/meta"
df = pd.read_excel(metadata_url, dtype='str')
df.fillna('', inplace=True)
df = df[:1000]

In [7]:
#LOAD DATA AS DF
#df = pd.read_csv(std_file)
#df = df[:100] #For testing
#df

## List of Script Functions:

In [8]:
# Order Geocode is the skeleton of the location cleaning process, it runs a number of 
# functions and provides some troubleshooting details should something go wrong
# It takes in th dataframe of values, a "stage" that lets users pick the geocoder, and a series of location
#inputs that tells the function which columns contain location data

def order_geocode(datafrm, stage, loc1, loc2="none", loc3="none", loc4="none"):
    if isinstance(datafrm, pd.DataFrame):
        if loc2 != "none" and loc2 not in list(datafrm.columns):
            print("Error loc2 is an invalid column name!")
        if loc3 != "none" and loc3 not in list(datafrm.columns):
            print("Error loc3 is an invalid column name!")
        if loc4 != "none" and loc4 not in list(datafrm.columns):
            print("Error loc4 is an invalid column name!")
        
        
        if loc1 in list(datafrm.columns):
            datafrm["location_series"] = datafrm[loc1]
            if loc2 != "none":
                datafrm["location_series"] = datafrm.location_series.map(str) + ", " + datafrm["loc2"].map(str)
            if loc3 != "none":
                datafrm["location_series"] = datafrm.location_series.map(str) + ", " + datafrm["loc3"].map(str)
            if loc4 != "none":
                datafrm["location_series"] = datafrm.location_series.map(str) + ", " + datafrm["loc4"].map(str)
                
            grouped_loc = datafrm.groupby("location_series").count()
            if stage == 1:
                holder1 = []
                c = 0
                for i in grouped_loc.index:
                    temp = str(do_geocode(text_cleaner(i)))
                    print(str(c) + ", ", end = '')
                    temp = temp.split(",")
                    holder2 = []
                    for k in temp:
                        holder2.insert(0,text_cleaner(k))
                    holder2 = post_clean(holder2)
                    holder1.append(holder2)
                    c = c + 1
                print(grouped_loc.shape)
                locs = column_selector(holder1,1)
                print(locs.shape)
                locs["location_series"] = grouped_loc.index
                
            if stage == 2:
                holder1 = []
                c = 0
                for i in grouped_loc.index:
                    temp = str(do_second_geocode(text_cleaner(i)))
                    print(str(c) + ", ", end = '')
                    temp = temp.split(",")
                    holder2 = []
                    for k in temp:
                        holder2.insert(0,text_cleaner(k))
                    holder2 = post_clean(holder2)
                    holder1.append(holder2)
                    c = c + 1
                print(grouped_loc.shape)
                locs = column_selector(holder1,1)
                print(locs.shape)
                locs["location_series"] = grouped_loc.index
                
            if stage == 3:
                holder1 = []
                c = 0
                for i in grouped_loc.index:
                    temp = str(do_third_geocode(text_cleaner(i)))
                    print(str(c) + ", ", end = '')
                    temp = temp.split(",")
                    holder2 = []
                    for k in temp:
                        holder2.insert(0,text_cleaner(k))
                    holder2 = post_clean(holder2)
                    holder1.append(holder2)
                    c = c + 1
                print(grouped_loc.shape)
                locs = column_selector(holder1,1)
                print(locs.shape)
                locs["location_series"] = grouped_loc.index
            return locs
                
        else:
            print("Loc1 must be a valid column name!")
    else:
        print("Datafrm must be a pandas dataframe type object!")
    

In [9]:
#Column selector allows for the dataframe columns to be dynamically resized when long addresses are returned
# Takes in all geocoded locations and sizes a dataframe depending on the max number of location attributes

def column_selector(data,count):
    i = 0
    columns_adj = []
    while i < count:
        columns_adj.append("Admin" + str(i))
        i = i + 1
    try:
        hold = pd.DataFrame(data,columns = columns_adj)
        try:
            hold = hold.rename(columns = {"Admin0":"Country"})
            return hold
        except:
            hold = hold.rename(columns = {"Admin0":"Country"})
            return hold
    except:
        return column_selector(data,count + 1)

In [10]:
#Cleans geocoded locations that return duplicates and numeric addresses
#Takes in (Row) the geocoded location as a list of address attributes


def post_clean(row):
    good_values = []
    num_holder = ""
    temp_row = row.copy()
    
    for i in row:

        temp_row.remove(i)

        if i in temp_row:
            break
        elif num_holder != "":
            i = num_holder + " " + i

        try:
            temp = int(i)
        except ValueError:
            num_holder = ""
            good_values.append(i)
        else:
            num_holder = str(temp)

    return good_values              


In [11]:
#Second Geocoder, good option with no limit however can fail to geocode locations, used as a backup option
#Takes in an address

geolocator = Nominatim(user_agent="covid19_geocoding")
from geopy.exc import GeocoderTimedOut

#Geocodes
def do_geocode(address):
    try:
        return geolocator.geocode(address, timeout = 5, language = "en-US")
    except GeocoderTimedOut:
        return do_geocode(address)

In [12]:
do_geocode("California / Grand Princess cruise ship")

In [13]:
#Geonames geocoder 1000 hourly limit 20000 daily limit, primary geocoder because of hight accuracy
#Takes in an address

geolocator = GeoNames(user_agent="covid19_geocoding", username ="bdriggs599")
from geopy.exc import GeocoderTimedOut

#Geocodes
def do_second_geocode(address):
    try:
        return geolocator.geocode(address, exactly_one=True, timeout = 30)
    except GeocoderTimedOut:
        return do_geocode(address)

In [14]:
#Photon Geocoder, slow and limited to 2500 a day, only used for twice failed geocodes
#Takes in an address

geolocator = Photon(user_agent="covid19_geocoding", domain ="photon.komoot.de")
from geopy.exc import GeocoderTimedOut

#Geocodes
def do_third_geocode(address):
    try:
        return geolocator.geocode(address, exactly_one=True, timeout = 5)
    except GeocoderTimedOut:
        return do_geocode(address)

In [15]:
#Function for standarizing unique charecters and symbols which can cause location mismatches.
#Takes string of text

def text_cleaner(text):
    text = str(text).lower()
    text = text.strip()
    text = text.replace("-"," ")
    text = text.replace("_"," ")
    text = text.replace(","," ")
    text = text.replace("!","")
    text = text.replace("ä","a")
    text = text.replace("ó","o")
    text = text.replace("è","e")
    text = text.replace(":",",")
    text = text.replace("í","i")
    text = text.replace("ö","o")
    text = text.replace("á","a")
    text = text.replace("ú","u")
    text = text.replace("å","a")
    text = text.replace("é","e")
    text = text.replace("ô","o")
    return text

In [16]:
#Function for standarizing unique charecters and symbols which can cause location mismatches.
#Takes in a column of text
def column_cleaner(text):
    text = text.astype(str)
    text = text.replace('\d+', '')
    text = text.apply(lambda x: x.strip().lower())
    text = text.replace("-"," ")
    text = text.replace("_"," ")
    text = text.replace(","," ")
    text = text.replace("!","")
    text = text.replace("ä","a")
    text = text.replace("ó","o")
    text = text.replace("è","e")
    text = text.replace(":",",")
    text = text.replace("í","i")
    text = text.replace("ö","o")
    text = text.replace("á","a")
    text = text.replace("ú","u")
    text = text.replace("å","a")
    text = text.replace("é","e")
    text = text.replace("ô","o")
    return text

In [17]:
#function takes in two strings and returns a ratio of similarity
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

## Step 1: Initial Geocode

In [18]:
#DON'T rerun
df = pd.merge(df, order_geocode(df, 2, loc1), how='inner',left_on="location_series", right_on="location_series")
df.head(3)

0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, (221,

,Virus Strain Name,Accession ID,Data Source,Related ID,Nuc.Completeness,Sequence Length,Sequence Quality,Quality Assessment,Host,Sample Collection Date,Location,Originating Lab,Submission Date,Submitting Lab,Create Time,Last Update Time,location_series,Country,Admin1,Admin2,Admin3,Admin4
0,BetaCoV/Wuhan/HBCDC-HB-01/2019,NMDC60013088-01,NMDC,EPI_ISL_402132,Complete,29848,High,0/0/0/1/NO,Homo sapiens,2019-12-30,China / Hubei,Hubei Provincial Center for Disease Control an...,2020-01-19,Hubei Provincial Center for Disease Control an...,2020-01-20 20:04:48,2020-05-07 23:03:25,China / Hubei,china,hubei,None,None,None
1,BetaCoV/Nonthaburi/74/2020,EPI_ISL_403963,GISAID,,Complete,29859,High,0/0/0/0/NO,Homo sapiens,2020-01-13,Thailand/ Nonthaburi Province,"Department of Medical Sciences, Ministry of Pu...",2020-01-17,"Department of Medical Sciences, Ministry of Pu...",2020-01-20 20:04:48,2020-01-20 20:04:48,Thailand/ Nonthaburi Province,thailand,nonthaburi province,None,None,None
2,BetaCoV/Nonthaburi/61/2020,EPI_ISL_403962,GISAID,,Complete,29848,High,0/0/0/0/NO,Homo sapiens,2020-01-08,Thailand/ Nonthaburi Province,"Department of Medical Sciences, Ministry of Pu...",2020-01-17,"Department of Medical Sciences, Ministry of Pu...",2020-01-20 20:04:48,2020-01-20 20:04:48,Thailand/ Nonthaburi Province,thailand,nonthaburi province,None,None,None


In [19]:
#Save geocoded Info so function doesn't have to rerun
df.to_csv(std_file + "_GeoCoded", header=True, index=True)

## Step #2 Use GeoNames Geocoder to match failed geocodes

In [20]:
#Load back in saved data
df_country = pd.read_csv(std_file + "_GeoCoded",  index_col=0, low_memory=False)
df_country.shape

(1000, 22)

In [21]:
df_country

,Virus Strain Name,Accession ID,Data Source,Related ID,Nuc.Completeness,Sequence Length,Sequence Quality,Quality Assessment,Host,Sample Collection Date,Location,Originating Lab,Submission Date,Submitting Lab,Create Time,Last Update Time,location_series,Country,Admin1,Admin2,Admin3,Admin4
0,BetaCoV/Wuhan/HBCDC-HB-01/2019,NMDC60013088-01,NMDC,EPI_ISL_402132,Complete,29848,High,0/0/0/1/NO,Homo sapiens,2019-12-30,China / Hubei,Hubei Provincial Center for Disease Control an...,2020-01-19,Hubei Provincial Center for Disease Control an...,2020-01-20 20:04:48,2020-05-07 23:03:25,China / Hubei,china,hubei,NaN,NaN,NaN
1,BetaCoV/Nonthaburi/74/2020,EPI_ISL_403963,GISAID,NaN,Complete,29859,High,0/0/0/0/NO,Homo sapiens,2020-01-13,Thailand/ Nonthaburi Province,"Department of Medical Sciences, Ministry of Pu...",2020-01-17,"Department of Medical Sciences, Ministry of Pu...",2020-01-20 20:04:48,2020-01-20 20:04:48,Thailand/ Nonthaburi Province,thailand,nonthaburi province,NaN,NaN,NaN
2,BetaCoV/Nonthaburi/61/2020,EPI_ISL_403962,GISAID,NaN,Complete,29848,High,0/0/0/0/NO,Homo sapiens,2020-01-08,Thailand/ Nonthaburi Province,"Department of Medical Sciences, Ministry of Pu...",2020-01-17,"Department of Medical Sciences, Ministry of Pu...",2020-01-20 20:04:48,2020-01-20 20:04:48,Thailand/ Nonthaburi Province,thailand,nonthaburi province,NaN,NaN,NaN
3,BetaCoV/Wuhan/IVDC-HB-04/2020,NMDC60013085-01,NMDC,EPI_ISL_402120,Complete,29896,High,0/0/0/2/NO,Homo sapiens,2020-01-01,China / Hubei / Wuhan,National Institute for Viral Disease Control a...,2020-01-11,National Institute for Viral Disease Control a...,2020-01-20 20:04:48,2020-05-07 23:03:25,China / Hubei / Wuhan,china,hubei,wuhan city,NaN,NaN
4,BetaCoV/Wuhan/IVDC-HB-01/2019,NMDC60013084-01,NMDC,EPI_ISL_402119,Complete,29891,High,0/0/0/0/NO,Homo sapiens,2019-12-30,China / Hubei / Wuhan,National Institute for Viral Disease Control a...,2020-01-10,National Institute for Viral Disease Control a...,2020-01-20 20:04:48,2020-05-07 23:03:25,China / Hubei / Wuhan,china,hubei,wuhan city,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,hCoV-19/Peru/010/2020,MT263074,GenBank,EPI_ISL_415787,Partial,29856,High,0/0/NA/NA/NA,Homo sapiens,2020-03-10,Peru / Lima,Laboratorio de Referencia Nacional de Biotecno...,2020-03-30,Laboratorio de Referencia Nacional de Biotecno...,2020-03-20 15:46:36,2020-04-01 11:30:48,Peru / Lima,peru,NaN,NaN,NaN,NaN
996,hCoV-19/Brazil/SPBR-10/2020,EPI_ISL_416032,GISAID,,Complete,29867,High,0/0/0/3/NO,Homo sapiens,2020-03-04,Brazil / Distrito Federal / Brasilia,"Instituto Adolfo Lutz, Interdiciplinary Proced...",2020-03-20,"Instituto Adolfo Lutz, Interdiciplinary Proced...",2020-03-20 15:46:36,2020-03-20 15:46:36,Brazil / Distrito Federal / Brasilia,brazil,federal district,70.701 010,brasilia,NaN
997,hCoV-19/Denmark/SSI-102/2020,EPI_ISL_415647,GISAID,,Complete,29958,Low,101/1/NA/NA/NA,Homo sapiens,2020-03-03,Denmark / Copenhagen,Statens Serum Institute,2020-03-18,Statens Serum Institute,2020-03-21 12:30:47,2020-03-21 12:30:47,Denmark / Copenhagen,denmark,capital region of denmark,1357 copenhagen,NaN,NaN
998,hCoV-19/Denmark/SSI-05/2020,EPI_ISL_416140,GISAID,,Complete,29832,High,"0/0/0/11/28881~28883(3-3-1.00,SNP:28881; SNP:2...",Homo sapiens,2020-03-02,Denmark / Copenhagen,Statens Serum Institute,2020-03-20,Statens Serum Institute,2020-03-21 12:30:47,2020-03-21 12:30:47,Denmark / Copenhagen,denmark,capital region of denmark,1357 copenhagen,NaN,NaN


In [22]:
#Number of successful geocodes
good_geocodes = df_country[df_country["Country"] != "none"]
bad_geocodes = df_country[df_country["Country"] == "none"]
print("Num of successful geocodes: " + str(good_geocodes.shape[0]))
print("Num of unsuccessful geocodes: " + str(bad_geocodes.shape[0]))

Num of successful geocodes: 1000
Num of unsuccessful geocodes: 0


In [23]:
#Double and triple geocodes missing locations

if bad_geocodes.shape[0] == 0:
    print("No location geocode errors")
    df_temp = good_geocodes
    
else:
    print(bad_geocodes["location_series"].unique())
    #Only keep relevent columns
    bad_geocodes= bad_geocodes[['Virus Strain Name', 'Accession ID', 'Data Source', 'Related ID',
       'Nuc.Completeness', 'Sequence Length', 'Sequence Quality',
       'Quality Assessment', 'Host', 'Sample Collection Date', 'Location',
       'Originating Lab', 'Submission Date', 'Submitting Lab',
       'location_series']]
    second_geocode = order_geocode(bad_geocodes, 1 ,loc1)
    #Missed locations, an Improvement?
    temp = second_geocode[second_geocode["Country"] == "none"].shape[0]
    
    if temp == 0:
        print("Second geocode successful")
        second_geocode = second_geocode[second_geocode["Country"] != "none"]
        amended_locations = second_geocode
        bad_geocodes = df_country = pd.merge(bad_geocodes, amended_locations, how='left',left_on="location_series", right_on="location_series")
        df_temp = good_geocodes.append(bad_geocodes)
        
    else:
        print("Second geocode only unsuccessful on " + str(temp) + " Geocodes, changing to third geocode.")
        third_geocode = order_geocode(second_geocode[second_geocode["Country"] == "none"], 3 ,"location_series")
        second_geocode = second_geocode[second_geocode["Country"] != "none"]
        amended_locations = second_geocode.append(third_geocode)
        bad_geocodes = df_country = pd.merge(bad_geocodes, amended_locations, how='left',left_on="location_series", right_on="location_series")
        df_temp = good_geocodes.append(bad_geocodes)
        df_temp.shape
print("Geocoding completed, failed on " + str(bad_geocodes.groupby("location_series").count().shape[0]) + " geocodes")

No location geocode errors
Geocoding completed, failed on 0 geocodes


### Save failed geocodes in a csv file for manual correction:

In [24]:
temp = df_temp[df_temp["Country"] == "none"].groupby("location_series").count()
temp.to_csv(std_file + "_GeoCoded_Failures", header=True, index=True)

### Save full df in a csv file:

In [25]:
#Remove geocode failures
df_temp =  df_temp[df_temp["Country"] != "none"]
df_temp.to_csv(std_file + "_GeoCoded_Full", header=True, index=True)

## Step #3 Match on Geonames Data for Country

In [26]:
#Load back in saved data
df = pd.read_csv(std_file + "_GeoCoded_Full",  index_col=0, low_memory = False)
df.shape

(1000, 22)

In [27]:
#Load in Geoname country list for matching on our geocoded locations

countries = pd.read_csv("countryInfo.txt", sep='\t', skiprows=(np.arange(49))) #Skip metadata
countries = countries[["Country","#ISO","ISO3","ISO-Numeric","fips","geonameid","neighbours"]] #Filter for relevent data
countries = countries.rename(columns = {"neighbours" : "Neighbouring_Country"})
countries["Country"] = countries["Country"].str.lower()
countries.head(5)

,Country,#ISO,ISO3,ISO-Numeric,fips,geonameid,Neighbouring_Country
0,andorra,AD,AND,20,AN,3041565,"ES,FR"
1,united arab emirates,AE,ARE,784,AE,290557,"SA,OM"
2,afghanistan,AF,AFG,4,AF,1149361,"TM,CN,IR,TJ,PK,UZ"
3,antigua and barbuda,AG,ATG,28,AC,3576396,NaN
4,anguilla,AI,AIA,660,AV,3573511,NaN


In [28]:
#List locations with no geonames match, manually replace them

df = df.replace("nan", "none")
#None type entry
noner = pd.Series(["none","None","None","None","None","None","None"], index = ["Country","#ISO","ISO3","ISO-Numeric","fips","geonameid","Neighbouring_Country"])

#fixing countries without geonames matches
countries["Country"] = countries["Country"].replace({"czechia":"czech republic", "united states":"united states of america","netherlands":"the netherlands"})
df["Country"] = df["Country"].replace({"united states":"united states of america","czechia":"czech republic","netherlands":"the netherlands", "the gambia":"gambia", "brunei darussalam":"brunei"})
countries = countries.append(noner, ignore_index = True)


hold= []
for i in df["Country"].unique():
    temp = 0
    for j in countries["Country"].unique():
        if i == j:
            hold.append(i)
            temp = 1
            break
    if temp != 1: #USA is called united states and Czech Republic is called Czechia
        print(i)

nan


In [29]:
#Evaluate the change in data
print(df.shape)
df_country = pd.merge(df, countries, how='left',left_on="Country", right_on="Country")
df_country
print(df_country.shape)

(1000, 22)
(1000, 28)


In [30]:
df_country

,Virus Strain Name,Accession ID,Data Source,Related ID,Nuc.Completeness,Sequence Length,Sequence Quality,Quality Assessment,Host,Sample Collection Date,Location,Originating Lab,Submission Date,Submitting Lab,Create Time,Last Update Time,location_series,Country,Admin1,Admin2,Admin3,Admin4,#ISO,ISO3,ISO-Numeric,fips,geonameid,Neighbouring_Country
0,BetaCoV/Wuhan/HBCDC-HB-01/2019,NMDC60013088-01,NMDC,EPI_ISL_402132,Complete,29848,High,0/0/0/1/NO,Homo sapiens,2019-12-30,China / Hubei,Hubei Provincial Center for Disease Control an...,2020-01-19,Hubei Provincial Center for Disease Control an...,2020-01-20 20:04:48,2020-05-07 23:03:25,China / Hubei,china,hubei,NaN,NaN,NaN,CN,CHN,156,CH,1814991,"LA,BT,TJ,KZ,MN,AF,NP,MM,KG,PK,KP,RU,VN,IN"
1,BetaCoV/Nonthaburi/74/2020,EPI_ISL_403963,GISAID,NaN,Complete,29859,High,0/0/0/0/NO,Homo sapiens,2020-01-13,Thailand/ Nonthaburi Province,"Department of Medical Sciences, Ministry of Pu...",2020-01-17,"Department of Medical Sciences, Ministry of Pu...",2020-01-20 20:04:48,2020-01-20 20:04:48,Thailand/ Nonthaburi Province,thailand,nonthaburi province,NaN,NaN,NaN,TH,THA,764,TH,1605651,"LA,MM,KH,MY"
2,BetaCoV/Nonthaburi/61/2020,EPI_ISL_403962,GISAID,NaN,Complete,29848,High,0/0/0/0/NO,Homo sapiens,2020-01-08,Thailand/ Nonthaburi Province,"Department of Medical Sciences, Ministry of Pu...",2020-01-17,"Department of Medical Sciences, Ministry of Pu...",2020-01-20 20:04:48,2020-01-20 20:04:48,Thailand/ Nonthaburi Province,thailand,nonthaburi province,NaN,NaN,NaN,TH,THA,764,TH,1605651,"LA,MM,KH,MY"
3,BetaCoV/Wuhan/IVDC-HB-04/2020,NMDC60013085-01,NMDC,EPI_ISL_402120,Complete,29896,High,0/0/0/2/NO,Homo sapiens,2020-01-01,China / Hubei / Wuhan,National Institute for Viral Disease Control a...,2020-01-11,National Institute for Viral Disease Control a...,2020-01-20 20:04:48,2020-05-07 23:03:25,China / Hubei / Wuhan,china,hubei,wuhan city,NaN,NaN,CN,CHN,156,CH,1814991,"LA,BT,TJ,KZ,MN,AF,NP,MM,KG,PK,KP,RU,VN,IN"
4,BetaCoV/Wuhan/IVDC-HB-01/2019,NMDC60013084-01,NMDC,EPI_ISL_402119,Complete,29891,High,0/0/0/0/NO,Homo sapiens,2019-12-30,China / Hubei / Wuhan,National Institute for Viral Disease Control a...,2020-01-10,National Institute for Viral Disease Control a...,2020-01-20 20:04:48,2020-05-07 23:03:25,China / Hubei / Wuhan,china,hubei,wuhan city,NaN,NaN,CN,CHN,156,CH,1814991,"LA,BT,TJ,KZ,MN,AF,NP,MM,KG,PK,KP,RU,VN,IN"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,hCoV-19/Peru/010/2020,MT263074,GenBank,EPI_ISL_415787,Partial,29856,High,0/0/NA/NA/NA,Homo sapiens,2020-03-10,Peru / Lima,Laboratorio de Referencia Nacional de Biotecno...,2020-03-30,Laboratorio de Referencia Nacional de Biotecno...,2020-03-20 15:46:36,2020-04-01 11:30:48,Peru / Lima,peru,NaN,NaN,NaN,NaN,PE,PER,604,PE,3932488,"EC,CL,BO,BR,CO"
996,hCoV-19/Brazil/SPBR-10/2020,EPI_ISL_416032,GISAID,,Complete,29867,High,0/0/0/3/NO,Homo sapiens,2020-03-04,Brazil / Distrito Federal / Brasilia,"Instituto Adolfo Lutz, Interdiciplinary Proced...",2020-03-20,"Instituto Adolfo Lutz, Interdiciplinary Proced...",2020-03-20 15:46:36,2020-03-20 15:46:36,Brazil / Distrito Federal / Brasilia,brazil,federal district,70.701 010,brasilia,NaN,BR,BRA,76,BR,3469034,"SR,PE,BO,UY,GY,PY,GF,VE,CO,AR"
997,hCoV-19/Denmark/SSI-102/2020,EPI_ISL_415647,GISAID,,Complete,29958,Low,101/1/NA/NA/NA,Homo sapiens,2020-03-03,Denmark / Copenhagen,Statens Serum Institute,2020-03-18,Statens Serum Institute,2020-03-21 12:30:47,2020-03-21 12:30:47,Denmark / Copenhagen,denmark,capital region of denmark,1357 copenhagen,NaN,NaN,DK,DNK,208,DA,2623032,DE
998,hCoV-19/Denmark/SSI-05/2020,EPI_ISL_416140,GISAID,,Complete,29832,High,"0/0/0/11/28881~28883(3-3-1.00,SNP:28881; SNP:2...",Homo sapiens,2020-03-02,Denmark / Copenhagen,Statens Serum Institute,2020-03-20,Statens Serum Institute,2020-03-21 12:30:47,2020-03-21 12:30:47,Denmark / Copenhagen,denmark,capital region of denmark,1357 copenhagen,NaN,NaN,DK,DNK,208,DA,2623032,DE


### Save full df in a csv file:

In [31]:
df_country.to_csv(std_file + "_GeoNamed", header=True, index=True)

## Step #4 Match on GeoName Admin 1 level

In [32]:
#Load back in saved data
df_country = pd.read_csv(std_file + "_GeoNamed",  index_col=0, low_memory = False)
df_country.shape

(1000, 28)

In [33]:
#Load in admin level 1 info from Geonames.org
#Names in English for admin divisions. Columns: code, name, name ascii, geonameid

admin1 = pd.read_csv("admin1CodesASCII.txt", sep='\t', header= None)
admin1[["#ISO","State/Province Code"]] = pd.DataFrame([ x.split(".") for x in admin1[0].tolist()])
admin1 = admin1.rename(columns = {2 : "State/Province Name", 3:"Geonameid"})
admin1 = admin1.drop([0,1], axis =1)
admin1["State/Province Name"] = column_cleaner(admin1["State/Province Name"])
admin1.head()

,State/Province Name,Geonameid,#ISO,State/Province Code
0,sant julia de loria,3039162,AD,06
1,ordino,3039676,AD,05
2,la massana,3040131,AD,04
3,encamp,3040684,AD,03
4,canillo,3041203,AD,02


In [34]:
#Convert all relevent columns to standard strings
#Here we use column cleaner instead of individualy applying text_cleaner because it is much quicker
for k in df_country.columns[17:22]:
    df_country[k] = column_cleaner(df_country[k])
df_country.head(3)

,Virus Strain Name,Accession ID,Data Source,Related ID,Nuc.Completeness,Sequence Length,Sequence Quality,Quality Assessment,Host,Sample Collection Date,Location,Originating Lab,Submission Date,Submitting Lab,Create Time,Last Update Time,location_series,Country,Admin1,Admin2,Admin3,Admin4,#ISO,ISO3,ISO-Numeric,fips,geonameid,Neighbouring_Country
0,BetaCoV/Wuhan/HBCDC-HB-01/2019,NMDC60013088-01,NMDC,EPI_ISL_402132,Complete,29848,High,0/0/0/1/NO,Homo sapiens,2019-12-30,China / Hubei,Hubei Provincial Center for Disease Control an...,2020-01-19,Hubei Provincial Center for Disease Control an...,2020-01-20 20:04:48,2020-05-07 23:03:25,China / Hubei,china,hubei,nan,nan,nan,CN,CHN,156.0,CH,1814991.0,"LA,BT,TJ,KZ,MN,AF,NP,MM,KG,PK,KP,RU,VN,IN"
1,BetaCoV/Nonthaburi/74/2020,EPI_ISL_403963,GISAID,NaN,Complete,29859,High,0/0/0/0/NO,Homo sapiens,2020-01-13,Thailand/ Nonthaburi Province,"Department of Medical Sciences, Ministry of Pu...",2020-01-17,"Department of Medical Sciences, Ministry of Pu...",2020-01-20 20:04:48,2020-01-20 20:04:48,Thailand/ Nonthaburi Province,thailand,nonthaburi province,nan,nan,nan,TH,THA,764.0,TH,1605651.0,"LA,MM,KH,MY"
2,BetaCoV/Nonthaburi/61/2020,EPI_ISL_403962,GISAID,NaN,Complete,29848,High,0/0/0/0/NO,Homo sapiens,2020-01-08,Thailand/ Nonthaburi Province,"Department of Medical Sciences, Ministry of Pu...",2020-01-17,"Department of Medical Sciences, Ministry of Pu...",2020-01-20 20:04:48,2020-01-20 20:04:48,Thailand/ Nonthaburi Province,thailand,nonthaburi province,nan,nan,nan,TH,THA,764.0,TH,1605651.0,"LA,MM,KH,MY"


In [35]:
#This tool shows which columns have the most matched for admin1
#It also uses the matching strength specified to adjust the names of certain values
#Takes a long time to run
for k in df_country.columns[17:22]:
    hold= []
    for i in df_country[k].unique():
        temp = 1
        for j in admin1["State/Province Name"]:
            if matching_strength_on == True:
                hol = similar(i,j)
                if hol >= matching_strength:
                    df_country[k].replace(i,j)
                    temp = 0
                    break
            else:
                if i == j:
                    temp = 0
                    break
        if temp != 1:
            hold.append(i) #probably some issue with capitalization ect
    print("Column " + str(k) + " has " + str(len(hold)) + " matches") #Definitly a few wrong

Column Country has 4 matches
Column Admin1 has 62 matches
Column Admin2 has 6 matches
Column Admin3 has 3 matches
Column Admin4 has 1 matches


Now we can merge admin1 based off on the column matches above:

In [36]:
df_country["INDEX"] = df_country.index #To track rows that are kept in the merge
print(df_country.shape)
df_adm1_country = pd.merge(df_country, admin1, how='inner',left_on=["Country","#ISO"], right_on=["State/Province Name","#ISO"])
df_adm1_country
print(df_adm1_country.shape) #Evaluating loss
df_country = df_country.drop(df_country.index[df_adm1_country["INDEX"].unique()]) #drop successfully merged rows
print(df_country.shape)
df_adm1_country.head()

(1000, 29)
(11, 32)
(989, 29)


,Virus Strain Name,Accession ID,Data Source,Related ID,Nuc.Completeness,Sequence Length,Sequence Quality,Quality Assessment,Host,Sample Collection Date,Location,Originating Lab,Submission Date,Submitting Lab,Create Time,Last Update Time,location_series,Country,Admin1,Admin2,Admin3,Admin4,#ISO,ISO3,ISO-Numeric,fips,geonameid,Neighbouring_Country,INDEX,State/Province Name,Geonameid,State/Province Code
0,BetaCoV/Taiwan/2/2020,EPI_ISL_406031,GISAID,NaN,Complete,29878,High,0/0/0/4/NO,Homo sapiens,2020-01-23,China / Taiwan / Kaohsiung,Centers for Disease Control (Taiwan),2020-01-27,Centers for Disease Control (Taiwan),2020-01-29 12:53:40,2020-02-16 17:02:35,China / Taiwan / Kaohsiung,taiwan,kaohsiung,812 china steel,nan,nan,TW,TWN,158.0,TW,1668284.0,NaN,69,taiwan,7280291,04
1,Taiwan/NTU01/2020,MT066175,GenBank,EPI_ISL_408489,Complete,29870,High,0/0/0/2/NO,Homo sapiens,2020-01-31,China / Taiwan / Taipei,National Institute for Viral Disease Control &...,2020-02-09,National Institute for Viral Disease Control &...,2020-02-11 13:04:15,2020-02-16 17:02:35,China / Taiwan / Taipei,taiwan,taipei,bo'ai special zone,nan,nan,TW,TWN,158.0,TW,1668284.0,NaN,199,taiwan,7280291,04
2,BetaCov/Taiwan/NTU02/2020,MT066176,GenBank,EPI_ISL_410218,Complete,29870,High,0/0/0/2/NO,Homo sapiens,2020-02-05,China / Taiwan / Taipei,"Microbial Genomics Core Lab, National Taiwan U...",2020-02-13,"Microbial Genomics Core Lab, National Taiwan U...",2020-02-14 12:40:52,2020-02-15 15:47:58,China / Taiwan / Taipei,taiwan,taipei,bo'ai special zone,nan,nan,TW,TWN,158.0,TW,1668284.0,NaN,200,taiwan,7280291,04
3,BetaCoV/Taiwan/3/2020,EPI_ISL_411926,GISAID,NaN,Complete,29891,High,0/0/0/3/NO,Homo sapiens,2020-01-24,China / Taiwan / Taipei,Taiwan Centers for Disease Control,2020-02-21,Taiwan Centers for Disease Control,2020-02-22 14:01:53,2020-02-22 14:01:53,China / Taiwan / Taipei,taiwan,taipei,bo'ai special zone,nan,nan,TW,TWN,158.0,TW,1668284.0,NaN,201,taiwan,7280291,04
4,BetaCoV/Taiwan/4/2020,EPI_ISL_411927,GISAID,NaN,Complete,29890,High,0/0/0/0/NO,Homo sapiens,2020-01-28,China / Taiwan / Taipei,Taiwan Centers for Disease Control,2020-02-21,Taiwan Centers for Disease Control,2020-02-22 14:01:53,2020-02-22 14:01:53,China / Taiwan / Taipei,taiwan,taipei,bo'ai special zone,nan,nan,TW,TWN,158.0,TW,1668284.0,NaN,202,taiwan,7280291,04


In [37]:
print(df_country.shape)
df_adm1_adm1 = pd.merge(df_country, admin1, how='inner',left_on=["Admin1","#ISO"], right_on=["State/Province Name","#ISO"])
df_adm1_adm1
print(df_adm1_adm1.shape) #Evaluating loss
df_country = df_country.drop(df_country.INDEX[df_adm1_adm1["INDEX"].unique()])
print(df_country.shape)

(989, 29)
(566, 32)
(423, 29)


In [38]:
print(df_country.shape)
df_adm1_adm2 = pd.merge(df_country, admin1, how='inner',left_on=["Admin2","#ISO"], right_on=["State/Province Name","#ISO"])
df_adm1_adm2
print(df_adm1_adm2.shape) #Evaluating loss
df_country = df_country.drop(df_country.INDEX[df_adm1_adm2["INDEX"].unique()])
print(df_country.shape)

(423, 29)
(70, 32)
(353, 29)


In [39]:
print(df_country.shape)
df_adm1_adm3 = pd.merge(df_country, admin1, how='inner',left_on=["Admin3","#ISO"], right_on=["State/Province Name","#ISO"])
df_adm1_adm3
print(df_adm1_adm3.shape) #Evaluating loss
df_country = df_country.drop(df_country.INDEX[df_adm1_adm3["INDEX"].unique()])
print(df_country.shape)

(353, 29)
(2, 32)
(351, 29)


In [40]:
print(df_country.shape)
df_adm1_adm4 = pd.merge(df_country, admin1, how='inner',left_on=["Admin4","#ISO"], right_on=["State/Province Name","#ISO"])
df_adm1_adm4
print(df_adm1_adm4.shape) #Evaluating loss
df_country = df_country.drop(df_country.INDEX[df_adm1_adm4["INDEX"].unique()])
print(df_country.shape)

(351, 29)
(0, 32)
(351, 29)


In [41]:
#Array of sucessful matches
#Since we have tracked which rows were succesful matches we can merge them back together
#into one df with all of the rows that matched somewhere on adm1
df_adm1 = df_adm1_country.append(df_adm1_adm1, ignore_index = True).append(df_adm1_adm2, ignore_index = True).append(df_adm1_adm3, ignore_index = True).append(df_adm1_adm4, ignore_index = True)
df_adm1.shape

(649, 32)

In [42]:
#Join all data back together
#Now df containes rows that were able to match on geonames admin1 and the rows that weren't
print(df_country.shape)
cols = ['Virus Strain Name', 'Accession ID', 'Data Source', 'Related ID',
       'Nuc.Completeness', 'Sequence Length', 'Sequence Quality',
       'Quality Assessment', 'Host', 'Sample Collection Date', 'Location',
       'Originating Lab', 'Submission Date', 'Submitting Lab', 'Create Time',
       'Last Update Time', 'location_series', 'Country', 'Admin1', 'Admin2',
       'Admin3', 'Admin4', '#ISO', 'ISO3', 'ISO-Numeric',
       'fips', 'geonameid', 'Neighbouring_Country']
df = df_country.append(df_adm1)
print(df.shape)

(351, 29)
(1000, 32)


In [43]:
df = df.drop(["INDEX"], axis =1)

In [44]:
df.to_csv(std_file + "_GeoNamed_admin1", header=True, index=True)

## Step #5 Match on Geonames Admin2

In [45]:
#Load back in saved data
df_country = pd.read_csv(std_file + "_GeoNamed_admin1",  index_col=0, low_memory = False)
#df_country = df_country.drop(["INDEX_x","INDEX_y","INDEX"], axis =1)
df_country.shape

(1000, 31)

In [46]:
#Load in admin level 2 info from Geonames.org
#names for administrative subdivision 'admin2 code' (UTF8), 
#Format : concatenated codes <tab>name <tab> asciiname <tab> geonameId

admin2 = pd.read_csv("admin2Codes.txt", sep='\t', header= None)
admin2[["#ISO","State/Province Code", "Municipality Code"]] = pd.DataFrame([ x.split(".") for x in admin2[0].tolist()])
admin2 = admin2.rename(columns = {2 : "Municipality Name", 3:"Geonameid"})
admin2 = admin2.drop([0,1], axis =1)
admin2["Municipality Name"] = column_cleaner(admin2["Municipality Name"])
admin2.head()

,Municipality Name,Geonameid,#ISO,State/Province Code,Municipality Code
0,abu dhabi municipality,12047239,AE,01,101
1,al ain municipality,12047240,AE,01,102
2,al dhafra,12047241,AE,01,103
3,al fujairah municipality,12047242,AE,04,701
4,dibba al fujairah municipality,12047243,AE,04,702


In [47]:
#This tool shows which columns have the most matched for admin1
#It also uses the matching strength specified to adjust the names of certain values
#Takes a long time to run
for k in df_country.columns[17:22]:
    hold= []
    for i in df_country[k].unique():
        temp = 1
        for j in admin2["Municipality Name"]:
            if matching_strength_on == True:
                if type(i) == float:
                    i = str(i)
                if type(j) == float:
                    j = str(j)
                hol = similar(i,j)
                if hol >= matching_strength:
                    df_country[k].replace(i,j)
                    temp = 0
                    break
            else:
                if i == j:
                    temp = 0
                    break
        if temp != 1:
            hold.append(i) #probably some issue with capitalization ect
    print("Column " + str(k) + " has " + str(len(hold)) + " matches") #Definitly a few wrong

Column Country has 3 matches
Column Admin1 has 10 matches
Column Admin2 has 16 matches
Column Admin3 has 2 matches
Column Admin4 has 0 matches


In [48]:
df_country["INDEX"] = df_country.index #To track rows that are kept in the merge
print(df_country.shape)
df_adm1_country = pd.merge(df_country, admin2, how='inner',left_on=["Country","#ISO"], right_on=["Municipality Name","#ISO"])
df_adm1_country
print(df_adm1_country.shape) #Evaluating loss
df_country = df_country.drop(df_country.index[df_adm1_country["INDEX"].unique()]) #drop successfully merged rows
print(df_country.shape)
df_adm1_country.head()

(1000, 32)
(0, 36)
(1000, 32)


,Virus Strain Name,Accession ID,Data Source,Related ID,Nuc.Completeness,Sequence Length,Sequence Quality,Quality Assessment,Host,Sample Collection Date,Location,Originating Lab,Submission Date,Submitting Lab,Create Time,Last Update Time,location_series,Country,Admin1,Admin2,Admin3,Admin4,#ISO,ISO3,ISO-Numeric,fips,geonameid,Neighbouring_Country,State/Province Name,Geonameid_x,State/Province Code_x,INDEX,Municipality Name,Geonameid_y,State/Province Code_y,Municipality Code


In [49]:
print(df_country.shape)
df_adm1_adm1 = pd.merge(df_country, admin2, how='inner',left_on=["Admin1","#ISO"], right_on=["Municipality Name","#ISO"])
df_adm1_adm1
print(df_adm1_adm1.shape) #Evaluating loss
df_country = df_country.drop(df_country.INDEX[df_adm1_adm1["INDEX"].unique()])
print(df_country.shape)

(1000, 32)
(17, 36)
(978, 32)


In [50]:
print(df_country.shape)
df_adm1_adm2 = pd.merge(df_country, admin2, how='inner',left_on=["Admin2","#ISO"], right_on=["Municipality Name","#ISO"])
df_adm1_adm2
print(df_adm1_adm2.shape) #Evaluating loss
df_country = df_country.drop(df_country.INDEX[df_adm1_adm2["INDEX"].unique()])
print(df_country.shape)

(978, 32)
(36, 36)
(934, 32)


In [51]:
print(df_country.shape)
df_adm1_adm3 = pd.merge(df_country, admin2, how='inner',left_on=["Admin3","#ISO"], right_on=["Municipality Name","#ISO"])
df_adm1_adm3
print(df_adm1_adm3.shape) #Evaluating loss
df_country = df_country.drop(df_country.INDEX[df_adm1_adm3["INDEX"].unique()])
print(df_country.shape)

(934, 32)
(2, 36)
(931, 32)


In [52]:
print(df_country.shape)
df_adm1_adm4 = pd.merge(df_country, admin2, how='inner',left_on=["Admin4","#ISO"], right_on=["Municipality Name","#ISO"])
df_adm1_adm4
print(df_adm1_adm4.shape) #Evaluating loss
df_country = df_country.drop(df_country.INDEX[df_adm1_adm4["INDEX"].unique()])
print(df_country.shape)

(931, 32)
(0, 36)
(931, 32)


In [53]:
#Array of sucessful matches
#Since we have tracked which rows were succesful matches we can merge them back together
#into one df with all of the rows that matched somewhere on adm1
df_adm2 = df_adm1_country.append(df_adm1_adm1, ignore_index = True).append(df_adm1_adm2, ignore_index = True).append(df_adm1_adm3, ignore_index = True).append(df_adm1_adm4, ignore_index = True)
df_adm2.shape

(55, 36)

In [54]:
#Join all data back together
#Now df containes rows that were able to match on geonames admin1 and the rows that weren't
print(df_country.shape)
cols = ['Virus Strain Name', 'Accession ID', 'Data Source', 'Related ID',
       'Nuc.Completeness', 'Sequence Length', 'Sequence Quality',
       'Quality Assessment', 'Host', 'Sample Collection Date', 'Location',
       'Originating Lab', 'Submission Date', 'Submitting Lab', 'Create Time',
       'Last Update Time', 'location_series', 'Country', 'Admin1', 'Admin2',
       'Admin3', 'Admin4', '#ISO', 'ISO3', 'ISO-Numeric',
       'fips', 'geonameid', 'Neighbouring_Country', 'Municipality Name', "Municipality Code"]
df = df_country.append(df_adm2)
print(df.shape)

(931, 32)
(986, 38)


In [55]:
df = df[['Virus Strain Name', 'Accession ID', 'Data Source', 'Related ID',
       'Nuc.Completeness', 'Sequence Length', 'Sequence Quality',
       'Quality Assessment', 'Host', 'Sample Collection Date', 'Location',
       'Originating Lab', 'Submission Date', 'Submitting Lab', 'Create Time',
       'Last Update Time', 'location_series', 'Country', 'Admin1', 'Admin2',
       'Admin3', 'Admin4', '#ISO', 'ISO3', 'ISO-Numeric', 'fips', 'geonameid',
       'Neighbouring_Country', 'State/Province Name', 'Geonameid',
       'State/Province Code',
       'Municipality Name',
       'Municipality Code']]

In [56]:
df.to_csv(std_file + "_GeoNamed_admin2", header=True, index=True)


In [57]:
df

,Virus Strain Name,Accession ID,Data Source,Related ID,Nuc.Completeness,Sequence Length,Sequence Quality,Quality Assessment,Host,Sample Collection Date,Location,Originating Lab,Submission Date,Submitting Lab,Create Time,Last Update Time,location_series,Country,Admin1,Admin2,Admin3,Admin4,#ISO,ISO3,ISO-Numeric,fips,geonameid,Neighbouring_Country,State/Province Name,Geonameid,State/Province Code,Municipality Name,Municipality Code
43,BetaCoV/Kanagawa/1/2020,EPI_ISL_402126,GISAID,NaN,Partial,369,High,0/0/NA/NA/NA,Homo sapiens,2020-01-14,Japan / Kanagawa Prefecture,"Department of Virology III, National Institute...",2020-01-16,"Department of Virology III, National Institute...",2020-01-20 20:04:48,2020-01-20 20:04:48,Japan / Kanagawa Prefecture,japan,kanagawa prefecture,NaN,NaN,NaN,JP,JPN,392.0,JA,1861060.0,NaN,NaN,NaN,NaN,NaN,NaN
73,SI200040-SP,MN970003,GenBank,NaN,Partial,290,High,0/0/NA/NA/NA,Homo sapiens,2020-01-08,Thailand,"Faculty of Medicine, Chulalongkorn University",2020-01-19,"Faculty of Medicine, Chulalongkorn University",2020-01-29 17:48:45,2020-01-29 17:48:45,Thailand,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
74,SI200121-SP,MN970004,GenBank,NaN,Partial,290,High,0/0/NA/NA/NA,Homo sapiens,2020-01-13,Thailand,"Faculty of Medicine, Chulalongkorn University",2020-01-19,"Faculty of Medicine, Chulalongkorn University",2020-01-29 17:48:45,2020-01-29 17:48:45,Thailand,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
95,2019 nCoV/Italy-INMI1,MT008022,GenBank,EPI_ISL_406959,Partial,322,High,0/0/NA/NA/NA,Homo sapiens,2020-01,Italy / Rome,"Virology Laboratory, INMI L. Spallanzani",2020-02-01,"Virology Laboratory, INMI L. Spallanzani",2020-02-02 11:28:53,2020-02-01 10:37:20,Italy / Rome,italy,lazio,rome,NaN,NaN,IT,ITA,380.0,IT,3175395.0,"CH,VA,SI,SM,FR,AT",NaN,NaN,NaN,NaN,NaN
96,2019 nCoV/Italy-INMI2,MT008023,GenBank,EPI_ISL_406960,Partial,322,High,0/0/NA/NA/NA,Homo sapiens,2020-01,Italy / Rome,"Virology Laboratory, INMI L. Spallanzani",2020-02-01,"Virology Laboratory, INMI L. Spallanzani",2020-02-02 11:28:49,2020-02-01 10:37:20,Italy / Rome,italy,lazio,rome,NaN,NaN,IT,ITA,380.0,IT,3175395.0,"CH,VA,SI,SM,FR,AT",NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50,hCoV-19/USA/NY-NYUMC1/2020,EPI_ISL_414639,GISAID,,Complete,29903,Low,4694/0/NA/NA/NA,Homo sapiens,2020-03-04,United States / New York / Nassau County,"Departments of Pathology and Medicine, New Yor...",2020-03-15,"Departments of Pathology and Medicine, New Yor...",2020-03-16 13:36:12,2020-03-16 13:36:12,United States / New York / Nassau County,united states of america,new york,nassau county,NaN,NaN,US,USA,840.0,US,6252001.0,"CA,MX,CU",new york,NaN,NaN,nassau county,089
51,hCoV-19/USA/NY-NYUMC1/2020,EPI_ISL_414639,GISAID,,Complete,29903,Low,4694/0/NA/NA/NA,Homo sapiens,2020-03-04,United States / New York / Nassau County,"Departments of Pathology and Medicine, New Yor...",2020-03-15,"Departments of Pathology and Medicine, New Yor...",2020-03-16 13:36:12,2020-03-16 13:36:12,United States / New York / Nassau County,united states of america,new york,nassau county,NaN,NaN,US,USA,840.0,US,6252001.0,"CA,MX,CU",new york,NaN,NaN,nassau county,059
52,hCoV-19/Brazil/BA-312/2020,EPI_ISL_415105,GISAID,,Complete,29768,High,"3/0/0/9/28881~28883(3-3-1.00,SNP:28881; SNP:28...",Homo sapiens,2020-03-04,Brazil / Bahia / Feira de Santana,Instituto Oswaldo Cruz FIOCRUZ - Laboratory of...,2020-03-16,Instituto Oswaldo Cruz FIOCRUZ - Laboratory of...,2020-03-17 22:48:11,2020-03-17 22:48:11,Brazil / Bahia / Feira de Santana,brazil,bahia,feira de santana,NaN,NaN,BR,BRA,76.0,BR,3469034.0,"SR,PE,BO,UY,GY,PY,GF,VE,CO,AR",bahia,NaN,NaN,feira de santana,2910800
53,hCoV-19/Wales/PHW10/2020,EPI_ISL_415453,GISAID,,Partial,29611,Low,600/0/NA/NA/NA,Homo sapiens,2020-03-10,United Kingdom / Wales / Cardiff,Public Health Wales Microbiology Cardiff,2020-03-17,Public Health 

In [ ]:
#done